In [1]:
from sedona.spark import *
from pyspark.sql.functions import col, sum, avg, regexp_replace, broadcast, trim, format_number, concat, lit, round
from pyspark.sql import SparkSession
import time
from pyspark.sql.types import FloatType,IntegerType
import pandas as pd
import csv



spark = SparkSession.builder \
    .appName("Crime - Income GeoAnalysis") \
    .getOrCreate()

# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))




Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3424,application_1732639283265_3380,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None
Executor Memory: 4743M
Executor Cores: 2

In [2]:

sedona = SedonaContext.create(spark)


#spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "50MB")

geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
    .option("multiLine", "true").load(geojson_path) \
    .selectExpr("explode(features) as features") \
    .select("features.*")

flattened_df = blocks_df.select(
    [col(f"properties.{col_name}").alias(col_name) for col_name in
     blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
    .drop("properties") \
    .drop("type")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:

income_df = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv",
    header=True
).withColumnRenamed("Zip Code", "ZCTA10").withColumn(
    "Estimated_Median_Income",
    regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("double")
    ).drop("Community","Estimated Median Income")

crime_df = spark.read.csv(
    [
        "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
        "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
    ],
    header=True
).filter((col("LAT").isNotNull()) & (col("LON").isNotNull()))

valid_population_df = flattened_df.select("ZCTA10","COMM","POP_2010","HOUSING10","geometry").filter(
    (col("CITY") == "Los Angeles") &
    (col("ZCTA10") > 0) &
    ((col("POP_2010") > 0) & (col("HOUSING10") > 0)) &
    (trim(col("COMM")) != "")
    ).groupBy("ZCTA10","COMM").agg(sum("POP_2010").alias("Total Population"),sum("HOUSING10").alias("Total Housing"),ST_Union_Aggr("geometry").alias("geometry")
)

crime_with_geometry = crime_df.withColumn("geometry", ST_Point("LON", "LAT")).select("geometry", "Vict Descent") #Vict Descent for Query 4



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#explain what method it uses in the default join execution
start_time = time.time()

join_income_df = valid_population_df.join( 
    income_df,
    on="ZCTA10",
    how="inner"
).drop("ZCTA10") \
 .withColumn("Total Income", col("Total Housing") * col("Estimated_Median_Income")) \
 .drop("Estimated_Median_Income") \
 .groupBy("COMM").agg(
     sum("Total Population").alias("Total Population"),
     sum("Total Income").alias("Total Income"),
     ST_Union_Aggr("geometry").alias("geometry")
)

# Προσθήκη explain για το join_income_df
join_income_df.explain(True)

final_calculations_df = join_income_df.join(
    crime_with_geometry,
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
 .withColumn("Crimes per Person", format_number((col("Crime Count") / col("Total Population")), 5)) \
 .withColumn("Median Income Per Person", concat(lit("$"), format_number(round(col("Total Income") / col("Total Population")), 0))) \
 .drop("Total Income") \
 .orderBy("COMM")

# Προσθήκη explain για το final_calculations_df
final_calculations_df.explain(True)

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
'Aggregate ['COMM], ['COMM, sum('Total Population) AS Total Population#378, sum('Total Income) AS Total Income#380, st_union_aggr('geometry, org.apache.spark.sql.sedona_sql.expressions.ST_Union_Aggr@35e6bd11, class[value[0]: geometry], class[value[0]: array<geometry>], true, true, 0, 0, None) AS geometry#385]
+- Project [COMM#49, Total Population#292L, Total Housing#294L, geometry#299, Total Income#360]
   +- Project [COMM#49, Total Population#292L, Total Housing#294L, geometry#299, Estimated_Median_Income#199, (cast(Total Housing#294L as double) * Estimated_Median_Income#199) AS Total Income#360]
      +- Project [COMM#49, Total Population#292L, Total Housing#294L, geometry#299, Estimated_Median_Income#199]
         +- Project [ZCTA10#66, COMM#49, Total Population#292L, Total Housing#294L, geometry#299, Estimated_Median_Income#199]
            +- Join Inner, (ZCTA10#66 = ZCTA10#194)
               :- Aggregate [ZCTA10#66, COMM#49], [ZCTA10#66, COMM#49, sum(PO

In [4]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_replicate_nl").join(
    income_df.hint("shuffle_replicate_nl"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)
join_income_df.show(5)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+------------+--------------------+
|                COMM|Total Population|Total Income|            geometry|
+--------------------+----------------+------------+--------------------+
|     Adams-Normandie|            7842| 6.8942616E7|POLYGON ((-118.30...|
|              Alsace|           11728| 1.3181687E8|POLYGON ((-118.36...|
|Angeles National ...|              17|    562353.0|MULTIPOLYGON (((-...|
|    Angelino Heights|            2374|  4.374584E7|POLYGON ((-118.25...|
|              Arleta|           32876|3.98153976E8|POLYGON ((-118.42...|
+--------------------+----------------+------------+--------------------+
only showing top 5 rows

Time taken: 19.16 seconds

In [5]:
start_time = time.time()

final_calculations_df = join_income_df.hint("broadcast").join(
    crime_with_geometry.hint("broadcast"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [6]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
start_time = time.time()

join_income_df = valid_population_df.hint("merge").join(
    income_df.hint("merge"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("merge").join(
    crime_with_geometry.hint("merge"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "merge"  # Replace with the appropriate query ID
Q2 = "merge"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [8]:
parquet_s3_path = "s3://groups-bucket-dblab-905418150721/group17/query4_data.parquet"
final_calculations_df.write.mode("overwrite").parquet(parquet_s3_path)
print(f"Data saved to {parquet_s3_path}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data saved to s3://groups-bucket-dblab-905418150721/group17/query4_data.parquet

In [9]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
start_time = time.time()

join_income_df = valid_population_df.hint("merge").join(
    income_df.hint("merge"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("shuffle_hash").join(
    crime_with_geometry.hint("shuffle_hash"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "merge"  # Replace with the appropriate query ID
Q2 = "shuffle_hash"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [11]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
start_time = time.time()

join_income_df = valid_population_df.hint("merge").join(
    income_df.hint("merge"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("shuffle_replicate_nl").join(
    crime_with_geometry.hint("shuffle_replicate_nl"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "merge"  # Replace with the appropriate query ID
Q2 = "shuffle_replicate_nl"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [13]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
start_time = time.time()

join_income_df = valid_population_df.hint("merge").join(
    income_df.hint("merge"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("broadcast").join(
    crime_with_geometry.hint("broadcast"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "merge"  # Replace with the appropriate query ID
Q2 = "broadcast"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [15]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
start_time = time.time()

join_income_df = valid_population_df.hint("broadcast").join(
    income_df.hint("broadcast"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("broadcast").join(
    crime_with_geometry.hint("broadcast"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "broadcast"  # Replace with the appropriate query ID
Q2 = "broadcast"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [17]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
start_time = time.time()

join_income_df = valid_population_df.hint("broadcast").join(
    income_df.hint("broadcast"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("merge").join(
    crime_with_geometry.hint("merge"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "broadcast"  # Replace with the appropriate query ID
Q2 = "merge"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [19]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
start_time = time.time()

join_income_df = valid_population_df.hint("broadcast").join(
    income_df.hint("broadcast"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("shuffle_hash").join(
    crime_with_geometry.hint("shuffle_hash"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "broadcast"  # Replace with the appropriate query ID
Q2 = "shuffle_hash"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [21]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
start_time = time.time()

join_income_df = valid_population_df.hint("broadcast").join(
    income_df.hint("broadcast"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("shuffle_replicate_nl").join(
    crime_with_geometry.hint("shuffle_replicate_nl"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "broadcast"  # Replace with the appropriate query ID
Q2 = "shuffle_replicate_nl"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [23]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_hash").join(
    income_df.hint("shuffle_hash"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("shuffle_replicate_nl").join(
    crime_with_geometry.hint("shuffle_replicate_nl"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_hash"  # Replace with the appropriate query ID
Q2 = "shuffle_replicate_nl"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [25]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_hash").join(
    income_df.hint("shuffle_hash"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("shuffle_hash").join(
    crime_with_geometry.hint("shuffle_hash"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_hash"  # Replace with the appropriate query ID
Q2 = "shuffle_hash"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [27]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_hash").join(
    income_df.hint("shuffle_hash"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("broadcast").join(
    crime_with_geometry.hint("broadcast"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_hash"  # Replace with the appropriate query ID
Q2 = "broadcast"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [29]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_hash").join(
    income_df.hint("shuffle_hash"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("merge").join(
    crime_with_geometry.hint("merge"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_hash"  # Replace with the appropriate query ID
Q2 = "merge"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [31]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_replicate_nl").join(
    income_df.hint("shuffle_replicate_nl"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("merge").join(
    crime_with_geometry.hint("merge"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_replicate_nl"  # Replace with the appropriate query ID
Q2 = "merge"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [33]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_replicate_nl").join(
    income_df.hint("shuffle_replicate_nl"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("broadcast").join(
    crime_with_geometry.hint("broadcast"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_replicate_nl"  # Replace with the appropriate query ID
Q2 = "broadcast"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [35]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_replicate_nl").join(
    income_df.hint("shuffle_replicate_nl"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = join_income_df.hint("shuffle_hash").join(
    crime_with_geometry.hint("shuffle_hash"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_replicate_nl"  # Replace with the appropriate query ID
Q2 = "shuffle_hash"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [37]:
spark.catalog.clearCache()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
start_time = time.time()

join_income_df = valid_population_df.hint("shuffle_replicate_nl").join(
    income_df.hint("shuffle_replicate_nl"),
    on="ZCTA10",
    how="inner"
    ).drop("ZCTA10")\
    .withColumn("Total Income",col("Total Housing") * col("Estimated_Median_Income")
                ).drop("Estimated_Median_Income")\
    .groupBy("COMM").agg(
        sum("Total Population").alias("Total Population"),
        sum("Total Income").alias("Total Income"),
        ST_Union_Aggr("geometry").alias("geometry")
)

final_calculations_df = crime_with_geometry.hint("shuffle_replicate_nl").join(
    join_income_df.hint("shuffle_replicate_nl"),
    ST_Within(crime_with_geometry.geometry, join_income_df.geometry)
).groupBy(
    "COMM", 
    "Total Population", 
    "Total Income"
).count().withColumnRenamed("count", "Crime Count") \
    .withColumn("Crimes per Person",format_number((col("Crime Count")/col("Total Population")),5)) \
    .withColumn("Median Income Per Person",concat(lit("$"),format_number(round(col("Total Income") / col("Total Population")), 0))) \
    .drop("Total Income") \
    .orderBy("COMM")

final_df = final_calculations_df.select("COMM", "Median Income Per Person", "Crimes Per Person")
final_df.orderBy("COMM").show(30)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

log_file_path = "query_methods_time_log.csv"
Q1 = "shuffle_replicate_nl"  # Replace with the appropriate query ID
Q2 = "shuffle_replicate_nl"

# Append data to the file
with open(log_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([Q1, Q2, f"{elapsed_time:.2f}"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+-----------------+
|                COMM|Median Income Per Person|Crimes Per Person|
+--------------------+------------------------+-----------------+
|     Adams-Normandie|                  $8,791|          0.71487|
|              Alsace|                 $11,240|          0.54161|
|Angeles National ...|                 $33,080|          0.41176|
|    Angelino Heights|                 $18,427|          0.57329|
|              Arleta|                 $12,111|          0.42645|
|     Atwater Village|                 $28,481|          0.52883|
|       Baldwin Hills|                 $17,304|          0.99501|
|             Bel Air|                 $63,041|          0.39923|
|       Beverly Crest|                 $60,947|          0.36896|
|         Beverlywood|                 $29,268|          0.50850|
|       Boyle Heights|                  $8,494|          0.61719|
|           Brentwood|                 $60,847|          0.40586|
|         

In [39]:
log_file_path = "query_methods_time_log.csv"
log_df = pd.read_csv(log_file_path, header=None, names=["Q1", "Q2", "Elapsed Time (s)"])
print(log_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                      Q1                    Q2  Elapsed Time (s)
0                  merge                 merge             24.96
1                  merge          shuffle_hash             23.89
2                  merge  shuffle_replicate_nl             20.39
3                  merge             broadcast             20.21
4              broadcast             broadcast             21.71
5              broadcast                 merge             20.41
6              broadcast          shuffle_hash             19.71
7              broadcast  shuffle_replicate_nl             19.38
8           shuffle_hash  shuffle_replicate_nl             19.36
9           shuffle_hash          shuffle_hash             19.12
10          shuffle_hash             broadcast             19.76
11          shuffle_hash                 merge             19.23
12  shuffle_replicate_nl                 merge             19.56
13  shuffle_replicate_nl             broadcast             18.38
14  shuffle_replicate_nl 